In [51]:
import pandas as pd
from pandas import DataFrame
import numpy
import networkx as nx
import matplotlib.pyplot as plt
import os

from pyvis.network import Network

from collections import namedtuple

In [52]:
G = nx.Graph()
net = Network(height="750px", width="750px",
              bgcolor="#222222", font_color="white")

In [53]:
sheet_id = '1VJdFRq9w0POFoZKxncqSwhqkmjrbGfvC'
sheet_name = 'Sheet1'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

In [54]:
chartdata = pd.read_csv(url)
chartdata.dropna(axis=0, thresh=3, inplace=True)

In [55]:
wwc_players = chartdata['player'].where(
    chartdata['at_wwc'] == 'Y').fillna(value='NA')
nonwwc_players = chartdata['player'].where(
    chartdata['at_wwc'] == 'N').fillna(value='NA')
current_rel = chartdata['current_rel'].fillna(value='NA')
former_rel = chartdata['former_rel'].fillna(value='NA')

In [56]:
flag_icons = {
    "Australia": "icons\icons8-australia-48.png",
    "USA": "icons\icons8-usa-48.png",
    "Denmark": "icons\icons8-denmark-48.png",
    "Sweden": "icons\icons8-sweden-48.png",
    "Germany": "icons\icons8-germany-48.png",
    "England": "icons\icons8-england-48.png",
    "Netherlands": "icons\icons8-netherlands-48.png",
    "Scotland": "icons\icons8-scotland-48.png",
    "Ireland": "icons\icons8-ireland-48.png",
    "Switzerland": "icons\icons8-switzerland-48.png",
    "Brazil": "icons\icons8-brazil-48.png",
    "Norway": "icons\icons8-norway-48.png",
    "Spain": "icons\icons8-spain-48.png",
    "New Zealand": "icons\icons8-new-zealand-48.png",
    "Italy": "icons\icons8-italy-48.png",
    "Colombia": "icons\icons8-colombia-48.png"
}

player_country = chartdata['wwc_team']

In [57]:
images = {k: os.path.relpath(fname) for k, fname in flag_icons.items()}

In [58]:
# add wwc players
edge_data = zip(wwc_players, player_country)
for e in edge_data:
    plrs = e[0]
    ctry = e[1]

    if plrs == 'NA':
        continue

    G.add_node(plrs, title=plrs, color='#5a5af2',  shape='image',
               image=images[ctry], size=20, mass=.75)

# add non-wwc players
edge_data = zip(nonwwc_players, player_country)
for e in edge_data:
    plrs = e[0]
    ctry = e[1]

    if plrs == 'NA':
        continue

    G.add_node(plrs, color="#8f8fbc", title=plrs,
               shape='image', image=images[ctry])

edge_data = zip(wwc_players, current_rel, nonwwc_players)
# current relationships
for e in edge_data:

    wwcplrs = e[0]
    crel = e[1]
    nonplrs = e[2]
# in the relationship edges it firsts makes sure that there is data in crel -->
# if yes checks for data in wwcplrs --> then checks for nonplrs data
    if crel == 'NA':
        continue

    if wwcplrs != 'NA':
        G.add_edge(wwcplrs, crel, color='#0fbd66')

    if nonplrs != 'NA':
        G.add_edge(nonplrs, crel, color='#0fbd66')

# former relationships
edge_data = zip(wwc_players, former_rel, nonwwc_players)

for e in edge_data:
    wwcplrs = e[0]
    frel = e[1]
    nonplrs = e[2]

    if frel == 'NA':
        continue

    if wwcplrs != 'NA':
        G.add_edge(wwcplrs, frel, color='#5c7066')

    if nonplrs != 'NA':
        G.add_edge(nonplrs, frel, color='#5c7066')

In [59]:
neighbor_map = net.get_adj_list()

# add neighbor data to node hover data
for node in net.nodes:
    node["title"] += "\n\nRelationships: \n" + \
        "\n".join(neighbor_map[node["id"]])

In [60]:
net.from_nx(G)
net.show('nodes.html')